In [1]:
from typing import Any, Literal, Optional
from pathlib import Path

from pydantic import BaseModel
import pandas as pd

from scraping_kit import load_db_and_bots
from twitter45.params import (
    ArgsUserTimeline, ParamsUserTimeline
)

path_data = Path("data")
df_accounts = pd.read_excel(path_data / "twitter_accounts.xlsx")
db_tw, bots = load_db_and_bots(path_data, "scrape_tw")

Collection Names: ['raw', 'trends']
Bots: bots=[BotScraper(acc_name='Vera')]


In [2]:
class UserSuspended(BaseModel):
    status: Literal["suspended"]
    profile: str

class User(BaseModel):
    status: Literal["active"]
    profile: str
    rest_id: str
    blue_verified: bool
    avatar: str
    header_image: str
    desc: str
    name: str
    protected: Optional[bool]
    location: str
    friends: int
    sub_count: int
    id: str


In [3]:
screennames = df_accounts["screenName"].to_list()
screenname_test = screennames[0]
screenname_test

'crypto_pinapple'

In [4]:
params = ParamsUserTimeline(screenname=screenname_test, cursor="")
req_args = ArgsUserTimeline(params=params)
req_args

ArgsUserTimeline(method='GET', params=ParamsUserTimeline(screenname='crypto_pinapple', cursor=''))

In [5]:
idx_bot, bot = bots.random_bot()

In [6]:
insert_one_result_raw = db_tw.requests_and_save(req_args, bot)

In [8]:
db_tw.coll.raw.find_one({"endpoint_name": "user_timeline"})

{'_id': ObjectId('658dc1caea62d322f3dcaf52'),
 'creation_date': datetime.datetime(2023, 12, 28, 15, 43, 22, 562000),
 'req_args': {'method': 'GET',
  'params': {'screenname': 'crypto_pinapple', 'cursor': ''},
  'url': 'https://twitter-api45.p.rapidapi.com/timeline.php',
  'endpoint_name': 'user_timeline'},
 'response_json': {'pinned': {'tweet_id': '1575711934854230017',
   'bookmarks': 1,
   'created_at': 'Fri Sep 30 04:59:35 +0000 2022',
   'favorites': 68,
   'text': 'Life summed up so beautifully!!! https://t.co/ATteChVRU2',
   'lang': 'en',
   'views': None,
   'quotes': 0,
   'replies': 38,
   'retweets': 8,
   'conversation_id': '1575711934854230017',
   'media': {'photo': [{'media_url_https': 'https://pbs.twimg.com/media/Fd4NpXjaEAAWBoD.jpg'}]},
   'author': {'rest_id': '45851919',
    'name': 'Crypto !!',
    'screen_name': 'crypto_pinapple',
    'avatar': 'https://pbs.twimg.com/profile_images/1592767427544227840/REG5SbHT_normal.jpg',
    'blue_verified': False}},
  'timeline':

In [ ]:
response_json = response.json()

user = response_json.pop("user")
if user["status"] == "active":
    user = User(**user)
    db_tw.add_raw_data()
elif user["status"] == "suspended":
    user = UserSuspended(**user)
else:
    raise Exception("Opción inválida.")

display(user)

In [ ]:
class asd(BaseModel):
    pinned: Any
    timeline: Any
    next_cursor: Any
    user: Any

asd(**response.json()).user

In [ ]:
asd(**response.json()).pinned